In [ ]:
import os, glob, random
import numpy as np, pandas as pd
import librosa
import soundfile as sf

import joblib
from joblib import Parallel, delayed
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!mkdir -p /root/.kaggle/
!cp ../input/mykaggleapi/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mkdir -p "/tmp/RFCSAD_Wav_48k"
!mkdir -p "/tmp/RFCSAD_Wav_48k/train"
!mkdir -p "/tmp/RFCSAD_Wav_48k/test"

In [ ]:
TARGET_SR = 48000
SAVE_PATH = "/tmp/RFCSAD_Wav_48k"
train_audio_paths = glob.glob("../input/rfcx-species-audio-detection/train/*.flac")
test_audio_paths = glob.glob("../input/rfcx-species-audio-detection/test/*.flac")
print(len(train_audio_paths), len(test_audio_paths))

In [ ]:
def convert_to_wav(path):
    fname = path.split("/")[-1].split('.')[0]
    ftype = path.split("/")[-2]
    save_path = f"{SAVE_PATH}/{ftype}"
    
    y, sr = librosa.load(path, sr=TARGET_SR, mono=True, res_type="kaiser_fast")
    
    sf.write(f"{save_path}/{fname}.wav", y, samplerate=TARGET_SR)

In [ ]:
_ = Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(convert_to_wav)(path) for path in tqdm(train_audio_paths)
)

In [ ]:
_ = Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(convert_to_wav)(path) for path in tqdm(test_audio_paths)
)

In [ ]:
!cp "../input/rfcx-species-audio-detection/train_fp.csv" "/tmp/RFCSAD_Wav_48k/"
!cp "../input/rfcx-species-audio-detection/train_tp.csv" "/tmp/RFCSAD_Wav_48k/"
!cp "../input/rfcx-species-audio-detection/sample_submission.csv" "/tmp/RFCSAD_Wav_48k/"

In [ ]:
!ls "/tmp/RFCSAD_Wav_48k/"

In [ ]:
data = '''{
  "title": "RFCSAD-wav-48k-v1",
  "id": "gopidurgaprasad/RFCSAD-wav-48k-v1",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}
'''
text_file = open("/tmp/RFCSAD_Wav_48k/dataset-metadata.json", 'w+')
n = text_file.write(data)
text_file.close()

In [ ]:
!kaggle datasets create -p "/tmp/RFCSAD_Wav_48k/" --dir-mode zip